<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

https://web.stanford.edu/class/ee364b/lectures/primal_dual_subgrad_slides.pdf

https://www.cvxpy.org/examples/basic/quadratic_program.html

In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [ ]:
def gradient_maker(grads):

    from numpy import linalg as la

    def nearestPD(A):

        B = (A + A.T) / 2
        _, s, V = la.svd(B)

        H = np.dot(V.T, np.dot(np.diag(s), V))

        A2 = (B + H) / 2

        A3 = (A2 + A2.T) / 2

        if isPD(A3):
            return A3

        spacing = np.spacing(la.norm(A))
        
        I = np.eye(A.shape[0])
        k = 1
        while not isPD(A3):
            mineig = np.min(np.real(la.eigvals(A3)))
            A3 += I * (-mineig * k**2 + spacing)
            k += 1

        return A3


    def isPD(B):
        try:
            _ = la.cholesky(B)
            return True
        except la.LinAlgError:
            return False

    #
    G = grads.T
    n, d = grads.shape

    #
    A = G.T@G
    A_inv = la.inv(nearestPD(A))
    s = np.ones(n)
    g = G@A_inv@s
    g *= (np.mean(la.norm(G.T, axis=1))/la.norm(g))

    return g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [ ]:
shape = (2000, 7840)
grads = 1e-10*(1000*(np.random.randn(*shape)+1))*np.random.randn(*shape)

In [ ]:
g = gradient_maker(grads)

In [ ]:
np.min(grads@g)

3.0274451744517627e-12